PROJET MIDLCOIN:
Emilie Plagnet
Melissa Bourkdeche
Louis Dutheil


Il faut exécuter le tout, la simulations se trouve à la fin. Pour être un utilisateur qui peut intéragir c'est le 2.

In [ ]:

###################################   POINT ET FIELD ELEMENT   ###################################

class Point:

    def __init__(self, x, y, a, b):  # a et b paramètres de la courbe et (x,y) coordonnées
        self.a = a
        self.b = b
        self.x = x
        self.y = y
        if self.x!=None:
            if self.y ** 2 != self.x ** 3 + a * x + b:
                raise ValueError(
                '({},{}) pas sur la courbe'.format(x, y))  # permet de savoir si le point est bien sur la courbe

    def __eq__(self, other):
      #renvoie True si les points sont égaux(même paramètres de courbe et même coordonnées)
        return self.x == other.x and self.y == other.y and self.a == other.a and self.b == other.b

    def __ne__(self, other):
        return self.x != other.x or self.y != other.y or self.a != other.a or self.b != other.b

    # inverse de __eq__ (on aurait pu appeler __eq__)

    def __add__(self, other):
        # il y a 3 cas :
        # 0 un des deux points est le point infini
        # 1. le point est additionné à lui même
        # 2. P!=Q + pas avec la même abscisse
        # 3. P!=Q + même abscisse

        # cas0

        if self.x == None or self.y == None:
            x3 = other.x
            y3 = other.y
        # cas 1.2 other est le point infini
        elif other.x == None or other.y == None:
            x3 = self.x
            y3 = self.y

        elif self.__eq__(other):
            s = (3 * self.x ** 2 + self.a) / (2 * self.y)
            x3 = s ** 2 - 2 * self.x
            y3 = s * (self.x - x3) - self.y

        # cas 3
        elif self.x == other.x and self.y == (-1*other.y):
            x3 = None
            y3 = None

        else:

            x1 = self.x
            x2 = other.x
            y1 = self.y
            y2 = other.y
            s = (y2 - y1) / (x2 - x1)
            x3 = (s ** 2)-x1-x2
            y3 = s * (x1 - x3) - y1

        return self.__class__(x3, y3, self.a, self.b)



    def __repr__(self):
        if self.x == None:
            return "point infini"
        else:
            return "coord = {"+ str(self.x)+ ','+ str(self.y)+ "}"

    def __rmul__(self, coefficient): #cette fonction est prise du pdf
        coef = coefficient
        current = self
        result = self.__class__(None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result


#Prise du pdf, elle  permet d'avoir un champ finis mod P.
class FieldElement:

    def __init__(self, num, prime):
        if num >= prime or num < 0:
            error = 'Num {} not in field range 0 to {}'.format(num, prime - 1)
            raise ValueError(error)
        self.num = num
        self.prime = prime

    def __repr__(self):
        return 'FieldElement_{}({})'.format(self.prime, self.num)

    def __eq__(self, other):
        if other is None:
            return False
        return self.num == other.num and self.prime == other.prime

    def __ne__(self, other):
        # this should be the inverse of the == operator
        return not (self == other)

    def __add__(self, other):
        if self.prime != other.prime:
            raise TypeError('Cannot add two numbers in different Fields')
        # self.num and other.num are the actual values
        # self.prime is what we need to mod against
        num = (self.num + other.num) % self.prime
        # We return an element of the same class
        return self.__class__(num, self.prime)

    def __sub__(self, other):
        if self.prime != other.prime:
            raise TypeError('Cannot subtract two numbers in different Fields')
        # self.num and other.num are the actual values
        # self.prime is what we need to mod against
        num = (self.num - other.num) % self.prime
        # We return an element of the same class
        return self.__class__(num, self.prime)

    def __mul__(self, other):
        if self.prime != other.prime:
            raise TypeError('Cannot multiply two numbers in different Fields')
        # self.num and other.num are the actual values
        # self.prime is what we need to mod against
        num = (self.num * other.num) % self.prime
        # We return an element of the same class
        return self.__class__(num, self.prime)

    def __pow__(self, exponent):
        n = exponent % (self.prime - 1)
        num = pow(self.num, n, self.prime)
        return self.__class__(num, self.prime)

    def __truediv__(self, other):
        if self.prime != other.prime:
            raise TypeError('Cannot divide two numbers in different Fields')
        # self.num and other.num are the actual values
        # self.prime is what we need to mod against
        # use fermat's little theorem:
        # self.num**(p-1) % p == 1
        # this means:
        # 1/n == pow(n, p-2, p)
        num = (self.num * pow(other.num, self.prime - 2, self.prime)) % self.prime
        # We return an element of the same class
        return self.__class__(num, self.prime)

    def __rmul__(self, coefficient):
        num = (self.num * coefficient) % self.prime
        return self.__class__(num=num, prime=self.prime)
#Prise du pdf  permet d'être en 32bit avec le P qu'on a choisit
class S64Field(FieldElement):

    def __init__(self, num, prime=None):
        super().__init__(num=num, prime=P)

    def __repr__(self):
        return '{:x}'.format(self.num).zfill(8)

#NOS VALEUR P ET N: ordre du point génerateur, l'explication de notre choix est dans le pdf p.3
P=2**32-5
N=2147449079
#PRISE DU PDF
class S64Point(Point):

    def __init__(self, x, y, a=None, b=None):
        a, b = S64Field(1), S64Field(8)
        if type(x) == int:
            super().__init__(x=S64Field(x), y=S64Field(y), a=a, b=b)
        else:
            super().__init__(x=x, y=y, a=a, b=b)


    def __repr__(self):
        if self.x is None:
            return 'S64Point(infinity)'
        else:
            return 'S64Point({}, {})'.format(self.x, self.y)


    def __rmul__(self, coefficient):
        coef = coefficient % N
        return super().__rmul__(coef)

from random import randint

#POINT GENERATEUR ET ATTRIBUTS CHOISIT
gx = 1206299466
gy = 3144673629
a = 1
b = 8
pointGenerateur = S64Point(gx,gy)

#FOnction de hash, renvoie un entier en 32bit
def sha32(message):
    return int.from_bytes(hashlib.sha256(message.encode()).digest()[:4],'little')

#Classe clé privée
class ClePrivee:
    def __init__(self):
        self.clePrivee=randint(0,N)

#Classe clé publique
class ClePublique:
    def __init__(self,Cleprivee,nom):
        self.nom=nom
        self.trx=[]
        self.clePublique=Cleprivee.clePrivee*pointGenerateur
        self.blockchain=blockChain()

    def montantUTXO(self):
      #Unspent Transaction Output : furrycoins pas dépensés
        montant=0
        for transac in self.trx:
            for outp in transac.output:
                if outp.cleP.clePublique==self.clePublique:
                    montant+=outp.montant
        return montant


In [ ]:
###################################   SIGNATURE   ###################################
class Signature:
  def __init__(self,privateK,message) :
    self.message=message
    self.nonce=randint(0,N-1)
    self.messageHache=sha32(message) #little endian
    calcul=self.nonce*pointGenerateur
    #CREATION DE LA SIGNATURE
    self.x= calcul.x.num % N
    self.y= pow(self.nonce,N-2,N)*(self.messageHache +
                                privateK.clePrivee * self.x )%N

  def __repr__(self) :
    return 'Signature({:x},{:y})'.format(self.x,self.y)

  def verif(self, cPublique, message):  # Fonction de verification de la signature
    messageHache = sha32(message)
    if not ((self.x) < P - 1 and self.x >= 1 and self.y < P - 1 and (self.y) >= 1):  # car si =P =0
      return False

    inv = pow(self.y, N - 2, N) #inverse de y dans Z/NZ d'où la necessité d'avoir N premier, voir page 3
    u = (messageHache * inv) % N
    v = (self.x * inv) % N
    total = u * pointGenerateur + v * cPublique.clePublique
    return total.x.num % N == self.x

In [ ]:

###################################   TRANSACTIONS   ###################################

#classe des output ( sortie de transaction)
class Output:
    def __init__(self,cleP,montant):
        self.cleP=cleP
        self.montant=montant
        self.dispo=True  #indique si une sortie a deja été utilisé par le proprietaire comme input d'une nouvelle transaction, permet d'éviter que la fonction de création
        #d'input réutilise une transaction non depensée déjà utilisée qui est dans la file d'attente des mineurs. Sinon chaque utilisateur ne pourrait faire qu'une transaction par bloc

class Input:
    def __init__(self,prevTransaction):
        self.prevTransaction=prevTransaction

class Transaction:
    def __init__(self, A, B, clePubliqueExp):
        self.input = A
        self.output = B
        self.signature = None   #la transaction est créée puis signée
        self.id = str(self.creationIdentifiant())
        self.clePubliqueExp = clePubliqueExp

    def signerTransaction(self,clePrivee): #c'est celui qui créee la transaction qui signe
        self.signature=Signature(clePrivee,self.id)

    def creationIdentifiant(self): #permet de créer un identifiant toujours different grâce au randint
        if self.input==None:
            a=0
        else:
            a=len(self.input)
        return sha32(str(a+len(self.output)+randint(0,2^32)))

    #FONCTION DE VERIFICATION DE LA TRANSACTION :

    def VerifSignature(self):
        return self.signature.verif(self.clePubliqueExp,self.id)


    #VERIFICATION DU MONTANT : c'est a dire que dans les input, il y a autant d'argent que la somme des montants des outputs
    def VerifMontants(self):

        somme=0
        monnaie=0
        for i in self.input:
            trans=i.prevTransaction
            for output in trans.output:
                if output.cleP==self.clePubliqueExp:
                    somme+=output.montant
        for j in self.output:
            monnaie+=j.montant
        return monnaie==somme
    #VERIFICATION DU APPARTENANCE: vérifie si la transaction utilisée dans les input appartient bien au créateur de la nouvelle transaction
    def VerifAppartennance(self):
        valide=False
        for inp in self.input:
            prev=inp.prevTransaction
            for output in prev.output:
                if output.cleP==self.clePubliqueExp:
                    valide=True
            if not valide:
                return not valide
            valide=False
        return True

    #remet la variable dispo en True si une transaction n'a pas été validé par les mineurs
    def rendreOutputDispo(self):
        for output in self.output:
            output.dispo=True

    def ClePReceveur(self):
        for Output in self.output:
            if Output.cleP != self.clePubliqueExp:
                return Output.cleP


####  FONCTION DE CREATION D'UNE TRANSACTION  ###


def creationInput(clePublique,montant):
    if clePublique==None:  #il s'agit d'un achat alors on ne met pas d'input
      listeTransaction=[]
    else:
      listeTransaction=clePublique.trx
    InputTab=[]
    sommefinale=0
    i=0
    while sommefinale<montant and i<len(listeTransaction): #choisit les transactions non utilisée de la personne qui crée la transaction, si il n'y en a pas alors vide mais ne passera
    #pas les verifications
        listeOutput=listeTransaction[i].output
        #listeTransaction[i].output c'est une liste avec plusieurs output
        for output in listeOutput:

            if output.cleP == clePublique and output.dispo:   #on regarde que la transaction n'a pas été utilisée dans une transaction en attente de validation
                sommefinale+=output.montant
                output.dispo=False
                input=Input(listeTransaction[i])
                InputTab.append(input)
        i+=1
    return (InputTab,sommefinale)

#Creation des outputs
def creerOutput(CleDesti,cleExpediteur,montantAPayer,montantInput): #si il y a un reste,  on créait un nouvel output vers lui même dans la nouvelle transaction
    outputTab=[]
    outputTab.append(Output(CleDesti,montantAPayer))
    montantReste=montantInput-montantAPayer                 #reste à renvoyer à l'éxpediteur
    if montantReste > 0:
        if cleExpediteur!=None:
            outputTab.append(Output(cleExpediteur,montantReste))
    return outputTab


#Creation d'achat d'un utilisateur avec de l'argent réel d'où l'input en None
def creationAchat(CleDest,montantAchat):
    CleExp=None
    Input=None
    Output=creerOutput(CleDest,CleExp,montantAchat,0)
    print(CleDest.nom +" a acheté "+ str(montantAchat)+"FURRYCOIN")
    ##il ne reste rien puisque tout l'argent est reversé
    return Transaction(Input,Output,CleExp)

## Ces fonctions permettent au receveur d'une transactions d'avoir leur argent une fois qu'elle est validée et mise dans un bloc,
#il peut y avoir plusieurs outputs à une transaction alors on parcours une liste.
def recevoirAchat(transaction):
    for output in transaction.output:
        output.cleP.trx.append(transaction)
        print(output.cleP.nom+" a reçu " + str(output.montant) +" FURRYCOIN lors de la transaction de id :"+str(transaction.id))


def recevoirTransaction(transaction):
    cleExp = transaction.clePubliqueExp
    for input in transaction.input:
        cleExp.trx.remove(input.prevTransaction)

    print("\n-RECEPTION DE LA TRANSACTION :",transaction.id)
    for output in transaction.output:
        output.cleP.trx.append(transaction)
        print(output.cleP.nom+" reçoit "+str(output.montant)+ " FURRY COINS de la transaction id :"+str(transaction.id))

##fonction qui creait la transaction
def creationTransaction(CleDesti,cleExpediteur,montantAPayer):
  obj=creationInput(cleExpediteur,montantAPayer)
  montant=obj[1]
  transac=Transaction(obj[0],creerOutput(CleDesti,cleExpediteur,montantAPayer,montant),cleExpediteur)
  print("ID :"+transac.id+" "+cleExpediteur.nom + " crée une transaction de " + str(montantAPayer) + " pour " + CleDesti.nom)
  return transac


In [ ]:
global NB_TRANSAC_MAX
import hashlib


###################################   BLOCK ET BLOCKCHAIN   ###################################
class Block:

  def __init__(self,prev,listeTr):
    self.prev = prev #contient le bloc précedent
    self.nonce = None
    self.pdt = None
    self.listeTr = listeTr #liste des transactions
    self.lockTime = None #date de création du block
    self.mineur = None  #mineur qui a miné le block

  #renvoie l'identifiant du bloc
  def id(self):
    if self.pdt is not None :
      if self.prev is None :
        return str("000000000")+str(self.pdt)+str(self.single_hash())
      else :
        return str(self.prev.hash())+str(self.pdt)+str(self.single_hash())
    print("Erreur, pas encore de preuve de travail trouvé")

  def hash(self):
    return sha32(str((self.id())))

  def try_hash(self):
    return sha32(str(self.try_id()))

  def try_id(self):
    if self.prev is None :
      return str("000000000")+str(self.nonce)+str(self.single_hash())
    else :
      return str(self.prev.hash())+str(self.nonce)+str(self.single_hash())

  def single_hash(self):
    #renvoie un seul hash pour tous les hash des transactions
    single = ""
    for tr in self.listeTr :
      single+=str(tr)
    return str(sha32(single))


  def try_pdt(self,nonce,accuracy_pdt):
    #le hash du bloc en binaire doit finir par accuracy_pdt zéros
    self.nonce = nonce
    hash_block = self.try_hash()
    bin_block = bin(hash_block)[2:]
    len_block = len(bin_block)
    inv_bin_block = bin_block[len_block::-1] #on compte les bits à 0 en fin du binaire du hashage
    i = 0
    for bit in inv_bin_block :
      if i == accuracy_pdt:
        if self.pdt != nonce: #pour éviter de changer de pdt
          self.pdt = nonce
        return True
      elif bit != '0':
        return False
      else:
        i=i+1

  def add_tr(self,tr):
    #ajoute une transaction au bloc si c'est bien la transaction suivante (renvoie True si c'est le cas, False sinon)
    last_hash = self.listeTr[len(self.listeTr)-1]
    if str(last_hash) == str(tr.input[0].prevTransaction.id) :
      self.listeTr.append(tr.id)
      return True
    else :
      return False


class blockChain:
  def __init__(self):
    self.nbBlock = 0
    self.lastBlock = None #contient le dernier bloc de la blockChain

  def add_block(self,bloc):
    if self.lastBlock == bloc.prev : #vérifie que le bloc à ajouter contient bien en dernier bloc, le dernier bloc de la blockChain
      self.lastBlock=bloc
      self.nbBlock+=1
      print("Le bloc id :"+str(bloc.nonce)+" a été ajouté à la blockChain")
      return True
    else :
      print("ERREUR : le bloc id :" + str(bloc.nonce) + "n'a pas été ajouté")
      return False

In [ ]:
###################################   MINAGE   ###################################

NB_TRANSAC_MAX=5
class minage:
    def __init__(self,cleMineur):
        self.fileAttente=[]   #file des transactions en attente de validation
        self.clePuMineur=cleMineur


    #permet d'ajouter une transaction à la file d'attente du mineur
    def ajoutTransaction(self, trans):
        if trans.verifNoeud() == True:
            self.FileAttente.append(trans)
            print("la transaction est valide")
        else:
            print("La transaction n'est pas valide")

    def verifPaiement(self,block,halfing): #permet de recompenser le mineur
        if block.prev==self.clePuMineur.blockchain.lastBlock:
            creationAchat(self.clePuMineur, halfing)
            print("La réception du block a eu lieu pour tous les utilisateurs")
            return True
        else:
            print("Erreur lors de l'ajout du block")
            return False

    def verificationTransaction(self,transaction): #1 - signature #2 montant et #3 pas depensé
        if transaction.input==None: #cas de l'achat
            return True
        return transaction.VerifSignature() and transaction.VerifAppartennance() and transaction.VerifMontants()


    #fonction qui permet au mineur de calculer la preuve de travaille "pdt" en testant des valeurs jusqu'à avoir un hash qui finit par 10 zéros.
    def calcul_pdt(self,block,accuracy_pdt):
        ok=False
        i=0
        while not ok :
            i+=1
            ok = block.try_pdt(i,accuracy_pdt)

    #permet au mineur de creer un bloc
    def creationBlock(self,accuracy_pdt):
        if len(self.fileAttente) < NB_TRANSAC_MAX:
            return None
        else:
            try_block=Block(self.clePuMineur.blockchain.lastBlock,self.fileAttente[0:NB_TRANSAC_MAX]) #creation du bloc avec les 5 premieres transactions de la liste( se comporte comme une file)
            try_block.tr_hash=try_block.single_hash() #permet d'avoir l'équivalent de la racine de merkle pour pouvoir calculer la preuve de travaille en fonction des nos transactions
            self.calcul_pdt(try_block,accuracy_pdt) #calcul de la preuve de travaille

        return try_block


    def ajout_transaction_file(self,transaction): #verifie si la transaction est bonne et si oui, l'ajoute à la file d'attente du mineur
        if self.verificationTransaction(transaction):
            self.fileAttente.append(transaction)
            print("Le mineur :" +self.clePuMineur.nom+" valide et ajoute à sa file la transaction" +str(transaction.id))
        else:
          transaction.rendreOutputDispo()
          print("La transacton id :"+str(transaction.id) + " n'est pas validée par le mineur :" + self.clePuMineur.nom)


    def SuppressionListe(self): # permet de supprimer les transactions du dernier bloc de la file d'attente du mineur, pour qu'il ne puisse pas refaire une transaction déjà validé
        block=self.clePuMineur.blockchain.lastBlock
        for trans in block.listeTr:
            for transa in self.fileAttente:
                if trans.id==transa.id:
                    self.fileAttente.remove(transa)

In [ ]:
###################################   GESTION   ###################################
import time
import threading
import random

class utilisateur:
    def __init__(self,clePriv,nom):
        self.clePriv=clePriv
        self.nom=nom
        self.clePubl=ClePublique(clePriv,nom)

#Permet de stocker et transmettre les données aux utilisateurs
class serveur:
    def __init__(self):
        self.listeUtilisateur = []
        self.listeMineur = []
        self.halfing=10
        self.accuracy_pdt=10
        self.listeBlocMineurs=[]



    def creationUtilisateur(self,nom):
      #génerer une clé privée et clé publique donc créait un nouvel utilisateur
        user=utilisateur(ClePrivee(),nom)
        self.listeUtilisateur.append(user)
        return user


    def creationMineur(self,clePubl):
      #permet de rendre un utilisateur aussi mineur
        min=minage(clePubl)
        self.listeMineur.append(min)
        return min


    #réajuste la recompense tous les 3 blocks
    def reajusterHalfing(self,min):
        if (min.clePuMineur.blockchain.nbBlock) % 3 == 0 and min.clePuMineur.blockchain.nbBlock != 0:
            self.halfing = self.halfing / 2
            print("\n******* Réajustement de la récompense des mineurs *******\n")


    #fonctions de mises à jours
    def miseAjourListeMineurs(self):
        for mineur in self.listeMineur:
            mineur.SuppressionListe()

    def miseAjourBlockChain(self,block):
        for user in self.listeUtilisateur:
            user.clePubl.blockchain.add_block(block)

    def recepetionDesAchatsTransac(self,bloc):
        for trans in bloc.listeTr:
            if self.EstAchat(trans):
                recevoirAchat(trans)
            else:
                recevoirTransaction(trans)

    #permet d'appeller tous les mineurs qui minent en même temps à l'aide de thread
    def appelleMineurs(self):
        print("Tous les mineurs commencent à miner\n")
        listeThread=[]
        for min in self.listeMineur:

            listeThread.append(threading.Thread(target=self.minerCreerBlock,args=(min,)))
        for thread in listeThread:

            thread.start()
        for thread in listeThread:

            thread.join()
        return self.ajoutBlockBlockChain(self.comparaisonTemps((self.listeBlocMineurs))) #ajoute à la blockChain le bloc qui a été crée en premier



    def comparaisonTemps(self,listeBloc):
        plusPetit=listeBloc[0].lockTime
        max=listeBloc[0]
        for bloc in listeBloc:
            if bloc.lockTime < plusPetit:
                max=bloc
        return max

    def minerCreerBlock(self,min): #minage d'un bloc par un mineur
        print("\n")
        print(min.clePuMineur.nom+" commence à miner un block..\n")
        time.sleep(1)
        print("...\n")
        time.sleep(1)
        print("\n")
        time.sleep(random.random()) #permet que tt les mineurs ne finissent pas en même temps
        bloc= min.creationBlock(self.accuracy_pdt)
        bloc.lockTime=time.monotonic_ns()
        bloc.mineur=min

        self.listeBlocMineurs.append(bloc)

    #ajoute le bloc à la blockChain
    def ajoutBlockBlockChain(self,bloc):
        min=bloc.mineur
        valide = min.verifPaiement(bloc,self.halfing) #verif que le bloc est valide et paie le mineur
        time.sleep(0.4)
        print("\n")
        print("Le mineur le plus rapide est :"+bloc.mineur.clePuMineur.nom)
        if valide:
            print("\n")
            time.sleep(0.2)
            self.recepetionDesAchatsTransac(bloc)
            print("\n")
            print("En cours d'ajout à la BlockChain...")
            time.sleep(0.5)
            print("...\n")
            time.sleep(0.5)
            self.miseAjourBlockChain(bloc) #met à jour la blockChain de tous les utilisateurs
            self.reajusterHalfing(min)    #réajuste la recompense si besoin
            self.miseAjourListeMineurs()  # met à jour la file d'attente des mineurs
            print("Le block a bien été ajouté")
            self.listeBlocMineurs=[]
        else:
            print("Le block n'a pas été ajouté")

    def ilfautMiner(self): #indique si il y a besoin de miner, c'est à dire si la liste d'attente des mineurs contient plus de 5 transactions
        if len(self.listeMineur[0].fileAttente) >= 5:
            return self.appelleMineurs()
        else:
            return False


    def genererTransaction(self,user1,user2,montant):
        time.sleep(0.5)
        print("\n")
        transaction=creationTransaction(user2.clePubl, user1.clePubl, montant)
        transaction.creationIdentifiant()
        transaction.signerTransaction(user1.clePriv)
        for min in self.listeMineur:
            min.ajout_transaction_file(transaction)
        return self.ilfautMiner() #appelle ilfautMiner qui permet de dire au serveur s'il y a besoin de miner ou pas, donc permet la potientielle création d'un block


    def genererAchat(self,user,montant):
        time.sleep(0.5)
        print("\n")
        achat = creationAchat(user.clePubl, montant)
        achat.creationIdentifiant()
        achat.signerTransaction(user.clePriv)
        for min in self.listeMineur:
            min.ajout_transaction_file(achat)
        return self.ilfautMiner() #idem que genererTransaction

    def EstAchat(self,trans):
        return trans.input==None


In [ ]:
###################################   MENU  ###################################


#initalisation 2
def init_prog(Serveur):
    #USER
    user1 = Serveur.creationUtilisateur("Alice")
    user2 = Serveur.creationUtilisateur("Bob")
    user3 = Serveur.creationUtilisateur("Melissa")
    user4 = Serveur.creationUtilisateur("Damien")
    user5 = Serveur.creationUtilisateur("Martin")
    user_6 = Serveur.creationUtilisateur("Mineur 1")
    user_7 = Serveur.creationUtilisateur("Mineur 2")

    #MINEUR
    leMineur = Serveur.creationMineur(user_6.clePubl)
    Serveur.creationMineur(user5.clePubl)
    Serveur.creationMineur(user4.clePubl)


    #ACHAT
    achat1 = Serveur.genererAchat(user1, 100)
    achat2 = Serveur.genererAchat(user2, 100)
    achat3 = Serveur.genererAchat(user3, 100)
    achat4 = Serveur.genererAchat(user4, 100)
    achat5 = Serveur.genererAchat(user5, 100)


#initialisation 1
def simulation():
    Serveur=serveur()
     #USER
    user1 = Serveur.creationUtilisateur("Alice")
    user2 = Serveur.creationUtilisateur("Bob")
    user3 = Serveur.creationUtilisateur("Melissa")
    user4 = Serveur.creationUtilisateur("Damien")
    user5 = Serveur.creationUtilisateur("Martin")
    #MINEUR
    mineur1 = Serveur.creationMineur(user5.clePubl)
    mineur2 = Serveur.creationMineur(user3.clePubl)
    mineur3 = Serveur.creationMineur(user4.clePubl)
    #ACHAT
    achat1=Serveur.genererAchat(user1,100)
    achat2=Serveur.genererAchat(user2,100)
    achat3=Serveur.genererAchat(user3,100)
    achat4=Serveur.genererAchat(user4,100)
    achat4=Serveur.genererAchat(user4,100)
    #TRANSACTION
    trans1=Serveur.genererTransaction(user1,user2,5)
    trans2=Serveur.genererTransaction(user2, user3, 18)
    trans3=Serveur.genererTransaction(user4, user5, 50)
    trans4=Serveur.genererTransaction(user1, user3, 58)
    trans3=Serveur.genererTransaction(user4, user2, 5)
    trans6 = Serveur.genererAchat(user5, 1)


    visualiserBlockChain(Serveur,Serveur.listeUtilisateur[0])



#FONCTION AFFICHAGE DES TRANSACTIONS
def afficherOutput(transac):
    print("-Output: ")
    for sortie in transac.output:
        print("Destinataire :"+sortie.cleP.nom+ " ,Montant :"+str(sortie.montant))

def afficherInput(transac):
    print("-Input: ")
    for input in transac.input:
        print("Id de la precedent transac :" + str(input.prevTransaction.id))
def afficherTransac(transac,Serveur):
    if Serveur.EstAchat(transac):
        print("\n**** Achat de furryCoin: id: " + transac.id," ****")
        afficherOutput(transac)
    else:
        print("\n**** Transaction de furryCoin id: ", transac.id, "Expediteur",
              transac.clePubliqueExp.clePublique," ****")
        afficherInput(transac)
        afficherOutput(transac)

def afficherTransactionEnCours(leMineur,Serveur):
    print("#######################################\nListe des transactions en cours:") #du coup regarde la liste d'un mineur c pas tres clean
    if len(leMineur.fileAttente)==0:
        print("AUCUNE TRANSACTION EN ATTENTE\n\n")
    else:
        for transacEnAttente in leMineur.fileAttente:
            afficherTransac(transacEnAttente,Serveur)

def listeDesUtilisateurs(Serveur):
    print("\n################  Liste des utilisateurs  #######################")
    i=0
    for user in Serveur.listeUtilisateur:
        print(i,user.nom,":",user.clePubl.clePublique)
        i+=1

def consulterSesFonds(Serveur):
    print("NOMBRE DE FURRYCOIN :",Serveur.listeUtilisateur[0].clePubl.montantUTXO())
    print("Vos transactions :")
    if len(Serveur.listeUtilisateur[0].clePubl.trx) > 0:
        for transac in Serveur.listeUtilisateur[0].clePubl.trx:
            afficherTransac(transac,Serveur)
    else:
        print("IL Y EN A 0 !")


#permet à l'utilisateur principal( celui qui lance le programme ) de faire des transactions avec d'autres utilisateurs et d'acheter des bitcoins
def achatBitcoin(user_utilisateur,Serveur):
    print("Combien voulez-vous ?")
    montant=int(input()) #pas de verif faire gaffe
    return Serveur.genererAchat(user_utilisateur,montant)


def faireTransaction(Serveur):
    choix=None
    listeDesUtilisateurs(Serveur)
    print("A qui voulez-vous envoyez l'argent :")
    while choix not in [str(i) for i in range(0,len(Serveur.listeUtilisateur))]:
        choix=input()
    user_destinataire=Serveur.listeUtilisateur[int(choix)]
    print("Combien voulez-vous envoyez ?")
    montant=int(input())
    Serveur.genererTransaction(Serveur.listeUtilisateur[0],user_destinataire,montant)




#GENERER TRANSAC ALEATOIREMENT
#genere aleatoirement des transactions valide ou non d'un montant maximum de 50e
def genererAleatoireTransac(Serveur): # il faut avoir au moins 2 users
    liste_user=Serveur.listeUtilisateur
    expediteur=liste_user[randint(1,len(liste_user)-1)]
    receveur=liste_user[randint(0,len(liste_user)-1)]
    while receveur==expediteur:
        receveur = liste_user[randint(0, len(liste_user)-1)]
    Serveur.genererTransaction(expediteur,receveur,randint(1,50))


def genererAleatoire(Serveur):
    print("Combien de transactions aléatoire voulez-vous generer ?")
    choix=int(input())
    for i in range(0,choix):
        genererAleatoireTransac(Serveur)

#FONCTION D'AFFICHAGE : VISUALISER BLOCK CHAIN
def CreateurEspaceNom(nom,comble):
    taillenom=len(nom)
    espace=comble-taillenom
    mot=nom
    for k in range(0,espace-1):
        mot+=" "
    return mot


def visualiserTransaction(transaction,Serveur):
    if Serveur.EstAchat(transaction):
        print("                         | Input:                 Output:        |")
        print("                         | FURRYCOIN              "+CreateurEspaceNom(transaction.ClePReceveur().nom,15)+" |")
    else:
        print("                         | Input:                   Output:      |")
        print("                         | "+CreateurEspaceNom(transaction.clePubliqueExp.nom,26)+CreateurEspaceNom(transaction.output[0].cleP.nom,14)+"|")
        if len(transaction.output)==2:
          print("                         |                          "+CreateurEspaceNom(transaction.output[1].cleP.nom,14)+"|")

def MontantOutput(transaction,clep):
    for output in transaction.output:
        if output.cleP!=clep:
            return output.montant



def visualiserBlock(bloc,Serveur,numBlock):
    print("\n")
    print("                            *****  Nonce du bloc", bloc.nonce, "  *****")
    print("                            ********** Bloc n°"+str(numBlock)+" ************")
    print("                         ----------------------------------------                        ")
    print("                         |                                       |                       ")

    for trans in bloc.listeTr:
        print("                         | Montant de la transaction : "+CreateurEspaceNom(str(MontantOutput(trans,trans.clePubliqueExp)),11)+"|")
        visualiserTransaction(trans,Serveur)
        print("                         |                                       |                       ")
    print("                         ----------------------------------------                        ")


def visualiserBlockChain(Serveur,user):
    print("#################### VISUALISATION BLOC CHAIN #################################")
    print("\n")
    print("                            ***** Récompense du mineur : " + str(Serveur.halfing) + " *****")

    bc=user.clePubl.blockchain
    print("                                     ***** Nombre de Bloc : "+str(bc.nbBlock)+" *****")
    block=bc.lastBlock
    if block ==None:
        print("BLOCKCHAIN VIDE\n")
    else:
        numBlock=bc.nbBlock
        while block !=None :
            visualiserBlock(block,Serveur,numBlock)
            block=block.prev
            numBlock-=1
    print("\n****** FIN DE L'AFFICHAGE ******")


#MENU 2
def menu_utilisateur():
    #INITIALISATION
    Serveur=serveur()
    choix=None
    user_utilisateur=Serveur.creationUtilisateur("vous")

    init_prog(Serveur) #créér 5 autres utilisateurs
    while choix!="miaou":
        choix=None
        while choix not in [str(i) for i in range(1,8)]:
            print("\n#################### Que voulez-vous faire  ? #######################")
            print("\n1)Achat de bitcoin\n2)Faire une transaction\n3)Voir les transactions en cours",
            "\n4)Consulter ses fonds\n5)Generer des transactions aléatoires\n6)Visualiser la blockChain\n7) EXIT ")
            choix = input()

        if choix=='1':
            achatBitcoin(user_utilisateur,Serveur)
        elif choix=='3':
            afficherTransactionEnCours(Serveur.listeMineur[0],Serveur)
        elif choix=='2':
            faireTransaction(Serveur)
        elif choix=='4':
            consulterSesFonds(Serveur)
        elif choix=='5':
            genererAleatoire(Serveur)
        elif choix=='6':
            visualiserBlockChain(Serveur,Serveur.listeUtilisateur[0])
        elif choix=='7':
            print("FIN")
            return

#MENU PRINCIPAL
def menu():
    chat_ascii_art = r"""
     /\_/\
    ( o.o )
     > ^ <
    """
    print(chat_ascii_art)
    print("Bonjour, bienvenue,\n Que souhaites-tu faire ?\n 1) Simulation\n 2) Création d'un compte")
    reponse=input()
    while reponse!='1' and reponse !='2':
        print("Saisi erronée, retour au menu. ")
        print("Bonjour, bienvenu dans FurryCoin,\n Que souhaites tu fais ?\n 1) Simulation\n 2) Création d'un compte")
        reponse = input()
    if reponse=='1':
        simulation()
    else:
        menu_utilisateur()
        return

menu()


     /\_/\
    ( o.o )
     > ^ <
    
Bonjour, bienvenue,
 Que souhaites-tu faire ?
 1) Simulation
 2) Création d'un compte
2


Alice a acheté 100FURRYCOIN
Le mineur :Mineur 1 valide et ajoute à sa file la transaction2004296267
Le mineur :Martin valide et ajoute à sa file la transaction2004296267
Le mineur :Damien valide et ajoute à sa file la transaction2004296267


Bob a acheté 100FURRYCOIN
Le mineur :Mineur 1 valide et ajoute à sa file la transaction2098343407
Le mineur :Martin valide et ajoute à sa file la transaction2098343407
Le mineur :Damien valide et ajoute à sa file la transaction2098343407


Melissa a acheté 100FURRYCOIN
Le mineur :Mineur 1 valide et ajoute à sa file la transaction640403535
Le mineur :Martin valide et ajoute à sa file la transaction640403535
Le mineur :Damien valide et ajoute à sa file la transaction640403535


Damien a acheté 100FURRYCOIN
Le mineur :Mineur 1 valide et ajoute à sa file la transaction979268564
Le mineur :Martin valide et ajoute à sa file la